In [ ]:
!pip install transformers torch pandas openpyxl peft

In [ ]:
import os
import glob
import json
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from torch.optim import AdamW
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

In [ ]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

PyTorch Version: 2.9.0+cu126
CUDA Available: True


In [ ]:
print("Loading KoGPT2 model and tokenizer...")
MODEL_NAME = "skt/kogpt2-base-v2" #사용할 모델의 이름 저장

tokenizer = PreTrainedTokenizerFast.from_pretrained( #모델이 사용할 규칙을 토크나이저를 불러온다.
    MODEL_NAME,
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',#문장에 시작, 끝, 모르는 단어 처리
    pad_token='<pad>', mask_token='<mask>' #길이 맞춤, 학습을 할때 빈칸을 채운다.(실제 사용X)
)

base_model = GPT2LMHeadModel.from_pretrained(MODEL_NAME) #학습된 가중치 불러오기
base_model.resize_token_embeddings(len(tokenizer)) #입력층 크기를 토크나이저의 단어 수에 맞게 조정

print("Base model loaded.")

Loading KoGPT2 model and tokenizer...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Base model loaded.


In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 125,458,944 || trainable%: 0.2351


In [ ]:
class ChatbotDataset(Dataset):
    def __init__(self, file_path, tokenizer, dataset_type):
        self.tokenizer = tokenizer
        self.data = []
        self.dataset_type = dataset_type
        self.max_len = 256

        print(f"Loading data from: {file_path}")

        # [F]
        if self.dataset_type == "F_BOT": #봇 타입이 F일때 실행
            try: #에러 발생시 예외 처리
                print("Parsing as F-Bot (JSON)...")
                with open(file_path, "r", encoding="utf-8") as f: #지정된 경로에 json파일을 읽는다.
                    raw_data = json.load(f) #파일 내용을 처리할 수 있는 데이터로 변환해 변수에 저장

                if isinstance(raw_data, dict): #데이터가 {}인지 형태 확인
                    raw_data = raw_data.get('data', []) #{}이라면 data키 안의 실제 데이터를 꺼낸다.

                count = 0 #대화 쌍을 세기 위해서 0으로 초기화
                for item in tqdm(raw_data, desc="Processing Data"):
                    try: #예외 처리(데이터에 문제가 발생하면 다음 데이터로 넘어간다.)
                        if 'talk' not in item or 'content' not in item['talk']: #데이터에 talk나 content가 없으면 불량 판단
                            continue #불량이면 다음 데이터를 가져온다.

                        content = item['talk']['content'] #실제대화 내용을 content에 저장
                        i = 1 #대화 순서를 번호로 설정
                        while True: #한 대화 세션을 모두 찾는다.
                            user_key = f"HS0{i}" #대화 순서에 따른 사용자 질문 키를 만든다.
                            bot_key = f"SS0{i}" #대화 순서에 따른 챗봇 답변 키를 만든다.

                            if user_key in content and bot_key in content: #답변 키와 질문 키가 둘 다 존재하는지 확인
                                q = content[user_key] #실제 질문 내용를 저장
                                a = content[bot_key] #실제 답변 내용을 저장
                                if q and a: #실제 질문과 답변이 비어있지 않은지 확인
                                    formatted = f"Q: {q}\nA: {a}{tokenizer.eos_token}" #Q: 질문 \n A: 답변 </s>형식 변형
                                    self.data.append(formatted) #완성된 학습용 문자열을 최종 데이터 리스트에 추가
                                    count += 1 #카운터 증가
                                i += 1 #대화 순서 증가
                            else:
                                break #대화 턴이 없으면 종료
                    except:
                        continue

                print(f"Successfully loaded {count} dialogue pairs.")

            except Exception as e: #예외 처리
                print(f"Critical Error parsing F-Bot data: {e}") #오류 출력

        # [T]
        elif self.dataset_type == "T_BOT": #봇 타입이 T일때 실행
            search_pattern = os.path.join(file_path, "**", "*.json") #지정된 폴더에서 하위 폴더에 있는 모든 json파일 경로로 지정
            json_files = glob.glob(search_pattern, recursive=True) #모든 json파일을 찾아서 리스트로 저장
            for json_file in tqdm(json_files, desc="Parsing T-Bot files"): #파일 목록을 하나씩 꺼낸다.
                try: #예외 처리
                    with open(json_file, "r", encoding="utf-8") as f: #파일을 모두 읽기 모드로 연다.
                        raw_data = json.load(f) #json 내용을 데이터로 변환
                    if 'paragraph' in raw_data: #심리상담 데이터의 paragraph키가 있는지 확인
                        dialogue = raw_data['paragraph'] #대화 내용이 담긴 리스트를 가져온다.
                        for i in range(len(dialogue) - 1): #마지막 턴은 답변이 없으므로 -1을 한다.
                            curr = dialogue[i] #현재 대화
                            next_t = dialogue[i+1] #다음 대화
                            if curr.get('paragraph_speaker') == "내담자" and next_t.get('paragraph_speaker') == "상담사": #질문자가 내담자이고 답변자가 상담자인 경우만 추출
                                q = curr.get('paragraph_text') #내담자(질문) 추출
                                a = next_t.get('paragraph_text') #상담자(답변) 추출
                                if q and a: #실제 질문과 답변이 비어있지 않은지 확인
                                    formatted = f"Q: {q}\nA: {a}{tokenizer.eos_token}" #Q: 질문 \n A: 답변 </s>형식 변형
                                    self.data.append(formatted)#완성된 학습용 문자열을 최종 데이터 리스트에 추가
                except:
                    pass #파일에 에러가 나면 다음 파일로 넘어간다.

        print(f"Total processed data entries: {len(self.data)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        tokens = self.tokenizer(
            text, padding="max_length", truncation=True,
            max_length=self.max_len, return_tensors="pt"
        )
        return {
            "input_ids": tokens["input_ids"].squeeze(),
            "attention_mask": tokens["attention_mask"].squeeze()
        }

In [ ]:
def __len__(self):
        return len(self.data)

In [ ]:
def __getitem__(self, idx):
        text = self.data[idx]
        tokens = self.tokenizer(
            text, padding="max_length", truncation=True,
            max_length=self.max_len, return_tensors="pt"
        )
        return {
            "input_ids": tokens["input_ids"].squeeze(),
            "attention_mask": tokens["attention_mask"].squeeze()
        }

In [ ]:
BOT_TYPE = "F_BOT"   # F봇 학습 시
#BOT_TYPE = "T_BOT"   # T봇 학습 시

DRIVE_BASE_PATH = "/content/drive/My Drive/Dataset"
MODEL_SAVE_DIR = "/content/drive/My Drive/Dataset"

if BOT_TYPE == "F_BOT":
    DATA_FILE_PATH = os.path.join(DRIVE_BASE_PATH, "감성대화말뭉치(최종데이터)_Training.json")
    SAVE_MODEL_PATH = os.path.join(MODEL_SAVE_DIR, "kogpt2_F_bot_LoRA")
else:
    DATA_FILE_PATH = os.path.join(DRIVE_BASE_PATH, "라벨링데이터")
    SAVE_MODEL_PATH = os.path.join(MODEL_SAVE_DIR, "kogpt2_T_bot_LoRA")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 3e-5
epochs = 3
batch_size = 4

In [ ]:
def train(model, dataset, optimizer, device):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    model.to(device)

    print(f"Starting training for {BOT_TYPE}...")

    for epoch in range(epochs):
        model.train() #학습 모드 전환
        total_loss = 0 #손실 초기화
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}") #진행률 표시

        for batch in progress_bar: #데이터를 batch씩 꺼내와 학습 진행
            input_ids = batch['input_ids'].to(device) #질문과 답변이 숫자로 변환된 데이터를 GPU로 옮긴다.
            attention_mask = batch['attention_mask'].to(device)
            labels = input_ids.clone() #입력 데이터 복사본을 정답으로 사용

            optimizer.zero_grad() #기울기 초기화

            outputs = model( #순전파 진행
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss #오차 계산
            loss.backward() #역전파 진행
            optimizer.step() #기울기를 바탕으로 가중치를 업데이트

            total_loss += loss.item() #전체 오차를 구한다.
            progress_bar.set_postfix({'loss': loss.item()}) #진행률 표시

        avg_loss = total_loss / len(dataloader) #평균 오차 계산
        perplexity = torch.exp(torch.tensor(avg_loss)).item() #퍼플렉시티 계
        print(f"Epoch {epoch+1}/{epochs} | Avg Loss: {avg_loss:.4f} | Perplexity: {perplexity:.2f}")

    print("Training finished.")

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted.")
    os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
except ImportError:
    print("Not in Colab.")

try:
    dataset = ChatbotDataset(DATA_FILE_PATH, tokenizer, BOT_TYPE)

    if len(dataset) > 0:
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        train(model, dataset, optimizer, device)

        print(f"Saving LoRA model to {SAVE_MODEL_PATH}...")
        model.save_pretrained(SAVE_MODEL_PATH)
        tokenizer.save_pretrained(SAVE_MODEL_PATH)
        print("LoRA Model saved successfully!")
    else:
        print("ERROR: Dataset is empty. Check your file paths.")

except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
Loading data from: /content/drive/My Drive/Dataset/감성대화말뭉치(최종데이터)_Training.json
Parsing as F-Bot (JSON)...


Processing Data: 100%|██████████| 51628/51628 [00:00<00:00, 115508.98it/s]


Successfully loaded 145948 dialogue pairs.
Total processed data entries: 145948
Starting training for F_BOT...


Epoch 1/3: 100%|██████████| 36487/36487 [31:15<00:00, 19.46it/s, loss=0.371]


Epoch 1/3 | Avg Loss: 0.4074 | Perplexity: 1.50


Epoch 2/3: 100%|██████████| 36487/36487 [31:16<00:00, 19.45it/s, loss=0.4]


Epoch 2/3 | Avg Loss: 0.3624 | Perplexity: 1.44


Epoch 3/3: 100%|██████████| 36487/36487 [31:20<00:00, 19.40it/s, loss=0.331]


Epoch 3/3 | Avg Loss: 0.3585 | Perplexity: 1.43
Training finished.
Saving LoRA model to /content/drive/My Drive/Dataset/kogpt2_F_bot_LoRA...
LoRA Model saved successfully!


In [ ]:
def chat(model_path, user_input):
    if not os.path.exists(model_path):
        return "모델 파일이 없습니다."

    print("Loading LoRA model for chat...")

    base = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
    lora_model = PeftModel.from_pretrained(base, model_path)

    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
    lora_model.to(device)
    lora_model.eval()

    prompt = f"Q: {user_input}\nA:"
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        output = lora_model.generate(
            input_ids, max_length=128, pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id, do_sample=True,
            top_p=0.95, top_k=50, repetition_penalty=1.2
        )

    text = tokenizer.decode(output[0], skip_special_tokens=True)
    try:
        return text.split("A:")[1].strip()
    except:
        return text

In [ ]:
print(f"\n--- {BOT_TYPE} Chat Test ---")
try:
    test_inputs = [
        "오늘 너무 우울해서 아무것도 하기 싫어",
        "요즘 잠을 잘 못 자는 것 같아",
        "미래가 너무 불안해"
    ]

    for q in test_inputs:
        print(f"나: {q}")
        print(f"봇: {chat(SAVE_MODEL_PATH, q)}\n")

except Exception as e:
    print(f"Test Error: {e}")


--- F_BOT Chat Test ---
나: 오늘 너무 우울해서 아무것도 하기 싫어
Loading LoRA model for chat...
봇: 많이 힘드시겠어요. 그 기분을 어떻게 전할 수 있을까요?

나: 요즘 잠을 잘 못 자는 것 같아
Loading LoRA model for chat...
봇: 어떻게 하는 것이 가장 좋은 방법일까요?

나: 미래가 너무 불안해
Loading LoRA model for chat...
봇: 미래에 대해서 걱정이 많으시군요.



In [1]:
try:
    import peft
except ImportError:
    !pip install transformers torch peft

import torch
import os
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from peft import PeftModel
from google.colab import drive

# 2. 구글 드라이브 연결
try:
    drive.mount('/content/drive')
except:
    print("이미 연결되어 있거나 로컬 환경입니다.")

# 3. 설정
BASE_PATH = "/content/drive/My Drive/Dataset"
F_BOT_PATH = os.path.join(BASE_PATH, "kogpt2_F_bot_LoRA")
T_BOT_PATH = os.path.join(BASE_PATH, "kogpt2_T_bot_LoRA")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "skt/kogpt2-base-v2"

print("시스템 초기화 중... (베이스 모델 로딩)")

try:
    base_model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
    base_model.to(device)
    print("베이스 모델 준비 완료!")
except Exception as e:
    print(f"베이스 모델 로드 실패: {e}")
    exit()

def get_chatbot(mode):
    if mode == "F":
        target_path = F_BOT_PATH
        name = "F-Bot (공감형)"
    else:
        target_path = T_BOT_PATH
        name = "T-Bot (상담형)"

    if not os.path.exists(target_path):
        print(f"오류: {target_path} 경로에 모델이 없습니다.")
        return None, None

    print(f"\n[{name}] 로 장착 중...")
    model = PeftModel.from_pretrained(base_model, target_path)
    tokenizer = PreTrainedTokenizerFast.from_pretrained(target_path)
    model.eval()
    return model, tokenizer

def chat_system():
    print("\n" + "="*40)
    print("MBTI 성향 전환 챗봇 (F ↔ T)")
    print("명령어: '!변경'으로 성격 전환, '!종료'로 끝내기")
    print("="*40)

    current_mode = "F" #초가에는 F(공감형)로 설정
    model, tokenizer = get_chatbot(current_mode) #토그나이저를 불러온다.
    if model is None: return

    while True:
        bot_label = "F(공감)" if current_mode == "F" else "T(상담)" #F(공감)또는 T(상담)선택
        user_input = input(f"\n👤 나: ") #사용자가 입력한 내용을 저장

        if user_input.strip() in ["!종료", "!quit", "종료"]: #종료 명령어 입력시에 상담 종료
            print("대화를 종료합니다.")
            break

        if user_input.strip() in ["!변경", "!switch", "변경"]: #성격 변경 명령어 입력시 성격 변환
            current_mode = "T" if current_mode == "F" else "F"
            print(f"\n성격을 전환합니다! ({'F' if current_mode=='T' else 'T'} -> {current_mode})")
            model, tokenizer = get_chatbot(current_mode)
            continue

        if not user_input: continue

        prompt = f"Q: {user_input}\nA:" #학습 사용한 형식으로 변경해 답변 준비
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device) #질문을 숫자로 변환

        with torch.no_grad():
            output = model.generate(
                input_ids,
                max_length=256,
                min_length=32,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=True,
                top_k=50,
                top_p=0.92,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
                temperature=0.7
            )

        text = tokenizer.decode(output[0], skip_special_tokens=True) #숫자를 다시 텍스트로 변환
        try:
            response = text.split("A:")[-1].strip() #실제 답변만 잘라낸다.
        except:
            response = text

        response = response.replace("NAME", "사용자") #답변에 있는 단어를 교체
        response = response.replace("name", "사용자")
        response = response.replace("PLACE", "챗봇")
        response = response.replace("place", "챗봇")

        print(f"🤖 {bot_label} 봇: {response}") #답변 출력

if __name__ == "__main__":
    chat_system()

Mounted at /content/drive
시스템 초기화 중... (베이스 모델 로딩)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

베이스 모델 준비 완료!

MBTI 성향 전환 챗봇 (F ↔ T)
명령어: '!변경'으로 성격 전환, '!종료'로 끝내기

[F-Bot (공감형)] 로 장착 중...

👤 나: 나 힘들어
🤖 F(공감) 봇: 어떻게 해야 지금의 상황을 극복할 수 있을까요?  어머님께서 왜 그렇게 말씀하세요? 앞으로 어떤 노력을 하셨나요?

👤 나: 요즘 공부때문에 힘들어
🤖 F(공감) 봇: 어떻게 해야 하는지 구체적으로 말씀해 주실 수 있을까요? ? 뭘 하면 좋을까요!

👤 나: 공부때문에 스트에스를 받아
🤖 F(공감) 봇: 공부를 열심히 해야 해서 스트레스가 많으시군요. 어떻게 하면 좋을까요?

👤 나: !변경

성격을 전환합니다! (F -> T)

[T-Bot (상담형)] 로 장착 중...


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(



👤 나: 나 공부 때문에 스트에스를 받아
🤖 T(상담) 봇: 네, 네. 근데 이제 그게 어떤 스트레스를 받거나 이렇게 사용자 님한테 이걸 또 어떻게 해소해 주고 있는지 궁금하신 분들도 계시고요.

👤 나: 나 다이어트를 해야되는데 힘들어서 못하겠어
🤖 T(상담) 봇: 네, 네. 그러니까 이번에 이제 사용자 씨가 좀 많이 하셨던 것 같아요. 그래서 저도 조금 이렇게 좀 그런 생각이 들었거든요.

👤 나: 나 요즘 우울해서 어떻게 해야 할 지 모르겠어
🤖 T(상담) 봇: 너무 힘들어. 내가 뭔가 나를 좀 불안하게 만드는 거야. 내가 이거 안 해볼까, 지금?

👤 나: 요즘 너무 불안해
🤖 T(상담) 봇: 지금 불안하고 좀 긴장되고 좀 불안할 수 있는 상황인 것 같아요. 그런 상황을 좀 어떤 거를 통해서 이제 이렇게 좀 찾아보시고 어떻게 좀 이거를 내가 잘 대처할 수 있을까 이런 것들 한번 생각을 해 보시면 좋을 것 같아요?

👤 나: 요즘 너무 우울해
🤖 T(상담) 봇: 그러니까요? 사용자 님. 이게 이제 제가 이제 상담을 하다 보니까 사실 상담이 좀 이렇게 되게 좀 힘든 게 이게 우리 이제 그 상담이 이제 너무 힘들고 힘들거든요.

👤 나: 요즘 너무 술을 많이 먹어
🤖 T(상담) 봇: 그래요. 그럼 그게 이제 그게 내가 스트레스를 받으면 되게 좀 그게 이게 스트레스가 돼 가지고 뭔가 그렇게 좀 그런 것들을 좀 더 해야 되는데 그러지 못하시는 거죠?

👤 나: 담배를 못 끊겟어
🤖 T(상담) 봇: 그러니까 담배는 안 끊는다라고 말씀하셨을 때 사용자 님한테 물어봤더니 담배를 끊으세요?

👤 나: 술을 못끊겠어
🤖 T(상담) 봇: 그렇죠. 술이 없으세요? 술도 안하고. 그러면 내가 주정을 안하는 거예요.

👤 나: !변경

성격을 전환합니다! (T -> F)

[F-Bot (공감형)] 로 장착 중...

👤 나: 요즘 너무 불안해
🤖 F(공감) 봇: 어떻게 해야 불안을 해소할 수 있을까요?   뭔가 변화가 필요한가요?

👤 나: 머리가 아파
🤖 F(공감) 봇: 머